# 750k dataset 
- somebody call dora 


In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

#make plots leng 
sns.set(style="whitegrid")
%matplotlib inline

# load data 
df = pd.read_csv('/Users/eb2007/Library/CloudStorage/OneDrive-UniversityofCambridge/Documents/PhD/data/data_c4_raw.csv')

# initial data inspection 
print("Shape:", df.shape)
print("columns:", df.columns)
display(df.head())
print(df.info())
print(df.isnull().sum())

# basic stats 
display(df.describe(include='all'))

# visualise the data 
# distribution of target variable
# what are the values in columns
print("sample of diagnosis columns")
diagnosis_cols = [col for col in df.columns if 'diagnosis' in col]
print(diagnosis_cols)

# any autism diagnosis 
autism_cols = [col for col in df.columns if 'autism_diagnosis' in col]

df['autism_any'] = df[autism_cols].apply(
    lambda row: int(any(x in [1.0, 2.0, 3.0] for x in row if not pd.isnull(x))),
    axis=1
)

print(df['autism_any'].value_counts())
sns.countplot(x='autism_any', data=df)
plt.title('any autism diagnosis (0 no 1 yes)')
plt.show()

# multi class target: most specific autism subtype 
def get_first_autism_subtype(row):
    for x in row :
        if x in [1.0, 2.0, 3.0]:
            return int(x)
    return 0 # no autism diagnosis 

df['autism_subtype'] = df[autism_cols].apply(get_first_autism_subtype, axis=1)
print(df['autism_subtype'].value_counts())
sns.countplot(x='autism_subtype', data=df)
plt.title('Autism subtype')
plt.show()

#multi-label: one hot encoding for each subtype

#create seperate columns for each subtype 
for subtype in [1.0, 2.0, 3.0]:
    df[f'autism_subtype_{int(subtype)}'] = df[autism_cols].apply(
        lambda row: int(subtype in row.values), axis=1
    )

print(df[[f'autism_subtype_{i}' for i in [1, 2, 3]]].sum())


In [ ]:
# plotting distribution for each subtype 
for i in [1 ,2 ,3]:
    sns.countplot(x=f'autism_subtype_{i}', data=df)
    plt.title(f'Autism subtype {i} (0 no 1 yes)')
    plt.show()

# EDA

In [ ]:
# --- EDA feature overview and missing data ---

# list all columns and their types 
print(df.dtypes)

# count missing values per column 
missing = df.isnull().sum().sort_values(ascending=False)
print("Missing values per column:\n", missing[missing > 0])

# visualize missing data 
plt.figure(figsize=(12,6))
sns.heatmap(df.isnull(), cbar=False)
plt.title("Missing Data Heatmap")
plt.show()

In [ ]:
# --- Univariate Analysis: categorical features ---

# list of known coded categorical columns 
coded_cat_cols = ['sex', 'handedness', 'education', 'occupation', 'country_region']

for col in coded_cat_cols:
    if col in df.columns:
        plt.figure(figsize=(8,4))
        df[col].value_counts(dropna=False).sort_index().plot(kind='bar')
        plt.title(f'Values counts of {col}')
        plt.xlabel(f'{col} (coded)')
        plt.ylabel('Count')
        plt.show()

